In [ ]:
import pandas as pd
import os

from surprise import Dataset, Reader, SVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [8]:
# Set the path to the base folder containing the "example_kaggle/les_7/fashion_products" folder.
# Adjust the path as needed.
path = '/home/sergio/Desktop/DataScienceHW/example_kaggle/les_7'

# Build paths to the MovieLens CSV files
path = os.path.join(path, 'fashion_products')
fashion_file = os.path.join(path, 'fashion_products.csv')


In [10]:

data = pd.read_csv(fashion_file)
print(data.head())

   User ID  Product ID Product Name   Brand         Category  Price    Rating  \
0       19           1        Dress  Adidas    Men's Fashion     40  1.043159   
1       97           2        Shoes     H&M  Women's Fashion     82  4.026416   
2       25           3        Dress  Adidas  Women's Fashion     44  3.337938   
3       57           4        Shoes    Zara    Men's Fashion     23  1.049523   
4       79           5      T-shirt  Adidas    Men's Fashion     79  4.302773   

    Color Size  
0   Black   XL  
1   Black    L  
2  Yellow   XL  
3   White    S  
4   Black    M  
